Сгенерировать последовательности, которые состоят из цифр (от 0 до 9) и задаются следующим образом: x - последовательность цифр y1 = x1 yi = xi + x1 Если yi >= 10 то yi = yi - 10 Научить модель рекуррентной нейронной сети предсказывать yi по xi Использовать: RNN, LSTM, GRU

In [2]:
import numpy as np
import torch
import torch.nn as nn
from itertools import permutations
import random
import time
import torch.nn.functional as F
from torch.utils.data import DataLoader, TensorDataset

In [3]:
seq_length = 10

In [4]:
#x = random.sample(range(10), seq_length) 
#y = [(x[i] + x[0]) % 10 for i in range(seq_length)]

In [5]:
# генерация x, y
data_x = []
data_y = []
for k in range(100):
    x = random.sample(range(10), seq_length)
    y = [(x[i] + x[0]) % 10 for i in range(seq_length)]
    data_x.append(x)
    data_y.append(y)   
        

In [6]:
data_x[9]

[9, 2, 1, 3, 4, 0, 7, 8, 5, 6]

In [7]:
data_y[9]

[8, 1, 0, 2, 3, 9, 6, 7, 4, 5]

In [8]:
len(data_x)

100

In [9]:
len(data_y)

100

In [10]:
# делим на train/test
num_trn = int(0.75*len(data_x))                
X_trn, Y_trn = data_x[:num_trn], data_y[:num_trn]
X_test, Y_test = data_x[num_trn:], data_y[num_trn:]

In [11]:
len(X_trn), len(Y_trn)

(75, 75)

In [12]:
len(X_test), len(Y_test)

(25, 25)

In [13]:
X_tr = torch.tensor(X_trn).type(torch.FloatTensor)
Y_tr =  torch.tensor(Y_trn).type(torch.FloatTensor)

In [14]:
X_ts = torch.tensor(X_test).type(torch.FloatTensor)
Y_ts =  torch.tensor(Y_test).type(torch.FloatTensor)

In [15]:
X_tr.shape

torch.Size([75, 10])

RNN

In [17]:
class RNNModel(torch.nn.Module):
    
    def __init__(self):
        super(RNNModel, self).__init__()

        self.hidden_size = hidden_size
        self.input_size = input_size
        self.rnn = torch.nn.RNN(input_size, hidden_size, batch_first=True)
        self.linear = torch.nn.Linear(hidden_size, input_size)

    def forward(self, X, state=None):
        o, h = self.rnn(X)
        out = self.linear(o)
        return out

In [18]:
input_size=10
hidden_size=100


In [19]:
model = RNNModel()
#optimizer =torch.optim.Adam(model.parameters())
loss = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.001, momentum=0.01)
num_epochs = 20

In [20]:
for epoch in range(num_epochs):
    train_iters, train_passed  = 0, 0
    train_loss, train_acc = 0., 0.
    start=time.time()
    model.train()
    #обнуляем градиент
    optimizer.zero_grad()
    y_pred = model.forward(X_tr)
    l = loss(y_pred, Y_tr)
    l.backward()
    optimizer.step()

    train_loss += l.item()
    train_acc += (y_pred.argmax(dim=0) == Y_tr).sum().item()

    train_iters += 1
    train_passed += len(X_tr)
    
    
    test_iters, test_passed  = 0, 0
    test_loss, test_acc = 0., 0.
    model.eval()
    with torch.no_grad():
        pred = model(X_ts)
        l = loss(pred, Y_ts)
        test_loss += l.item()
        test_acc += (y_pred.argmax(dim=0)==Y_ts).sum().item()
        test_iters += 1
        test_passed += len(X_ts)
        
    print("epoch: {}, taked: {:.3f}, train_loss: {}, train_acc: {}, test_loss: {}, test_acc: {}".format(
        epoch, time.time() - start, round(train_loss / train_iters, 3), round(train_acc / train_passed, 3),
        round(test_loss / test_iters, 3), round(test_acc / test_passed, 3))
    )

epoch: 0, taked: 0.021, train_loss: 107.137, train_acc: 0.173, test_loss: 106.963, test_acc: 0.24
epoch: 1, taked: 0.012, train_loss: 105.879, train_acc: 0.173, test_loss: 106.269, test_acc: 0.24
epoch: 2, taked: 0.008, train_loss: 105.162, train_acc: 0.173, test_loss: 105.835, test_acc: 0.24
epoch: 3, taked: 0.006, train_loss: 104.724, train_acc: 0.173, test_loss: 105.546, test_acc: 0.24
epoch: 4, taked: 0.010, train_loss: 104.438, train_acc: 0.173, test_loss: 105.345, test_acc: 0.24
epoch: 5, taked: 0.008, train_loss: 104.241, train_acc: 0.173, test_loss: 105.2, test_acc: 0.24
epoch: 6, taked: 0.001, train_loss: 104.096, train_acc: 0.173, test_loss: 105.092, test_acc: 0.24
epoch: 7, taked: 0.000, train_loss: 103.985, train_acc: 0.173, test_loss: 105.008, test_acc: 0.24
epoch: 8, taked: 0.016, train_loss: 103.894, train_acc: 0.173, test_loss: 104.941, test_acc: 0.24
epoch: 9, taked: 0.000, train_loss: 103.816, train_acc: 0.173, test_loss: 104.886, test_acc: 0.24
epoch: 10, taked: 0.00

GRU

In [22]:
class SimpleGRU(torch.nn.Module):
    def __init__(self, input_size, hidden_size, num_layers):
        super(SimpleGRU, self).__init__()
        self.hidden_size  = hidden_size
        self.num_layers = num_layers
        self.gru = torch.nn.GRU(input_size=10, hidden_size=hidden_size, num_layers=num_layers,batch_first=True)
        self.fc1 = torch.nn.Linear(hidden_size, output_dim)
    
    def forward(self, x):
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size)
        
        out,_ = self.gru(x, h0)
        out = out.reshape(out.shape[0], -1)
        out = self.fc1(out)
        return out

In [23]:
input_size = 10
hidden_size = 100
num_layers = 1
output_dim = 10
model_gru = SimpleGRU(input_size, hidden_size, num_layers)
loss = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.001, momentum=0.01)

In [24]:
X_tr = X_tr.view(len(X_tr), 1, -1)
Y_tr = Y_tr.view(len(Y_tr), -1)

In [25]:
for epoch in range(num_epochs):
    train_iters, train_passed  = 0, 0
    train_loss, train_acc = 0., 0.
    start=time.time()
    model_gru.train()

    optimizer.zero_grad()
    y_pred = model_gru.forward(X_tr)
    l = loss(y_pred, Y_tr)
    l.backward()
    optimizer.step()

    train_loss += l.item()
    train_acc += (y_pred.argmax(dim=0) == Y_tr).sum().item()

    train_iters += 1
    train_passed += len(X_tr)
    
    
    test_iters, test_passed  = 0, 0
    test_loss, test_acc = 0., 0.
    model_gru.eval()
    with torch.no_grad():
        pred = model(X_ts)
        l = loss(pred, Y_ts)
        test_loss += l.item()
        test_acc += (y_pred.argmax(dim=0)==Y_ts).sum().item()
        test_iters += 1
        test_passed += len(X_ts)
        
    print("epoch: {}, taked: {:.3f}, train_loss: {}, train_acc: {}, test_loss: {}, test_acc: {}".format(
        epoch, time.time() - start, round(train_loss / train_iters, 3), round(train_acc / train_passed, 3),
        round(test_loss / test_iters, 3), round(test_acc / test_passed, 3))
    )

epoch: 0, taked: 0.008, train_loss: 104.54, train_acc: 0.213, test_loss: 104.606, test_acc: 0.2
epoch: 1, taked: 0.000, train_loss: 104.54, train_acc: 0.213, test_loss: 104.606, test_acc: 0.2
epoch: 2, taked: 0.000, train_loss: 104.54, train_acc: 0.213, test_loss: 104.606, test_acc: 0.2
epoch: 3, taked: 0.000, train_loss: 104.54, train_acc: 0.213, test_loss: 104.606, test_acc: 0.2
epoch: 4, taked: 0.000, train_loss: 104.54, train_acc: 0.213, test_loss: 104.606, test_acc: 0.2
epoch: 5, taked: 0.016, train_loss: 104.54, train_acc: 0.213, test_loss: 104.606, test_acc: 0.2
epoch: 6, taked: 0.000, train_loss: 104.54, train_acc: 0.213, test_loss: 104.606, test_acc: 0.2
epoch: 7, taked: 0.000, train_loss: 104.54, train_acc: 0.213, test_loss: 104.606, test_acc: 0.2
epoch: 8, taked: 0.000, train_loss: 104.54, train_acc: 0.213, test_loss: 104.606, test_acc: 0.2
epoch: 9, taked: 0.000, train_loss: 104.54, train_acc: 0.213, test_loss: 104.606, test_acc: 0.2
epoch: 10, taked: 0.000, train_loss: 104

LSTM

In [27]:
class LSTMModel(torch.nn.Module):
    def __init__(self, input_dim, hidden_dim, layer_dim):
        super(LSTMModel, self).__init__()
        
        # Hidden dimensions
        self.hidden_dim = hidden_dim
        # Number of hidden layers
        self.layer_dim = layer_dim
        # LSTM
        self.lstm = torch.nn.LSTM(input_dim, hidden_dim, layer_dim, batch_first=True) # batch_first=True (batch_dim, seq_dim, feature_dim)
        # Readout layer
        self.fc = torch.nn.Linear(hidden_dim, output_dim)

    def forward(self, x):
        x, _ = self.lstm(x)
        x = self.fc(x)
        return x
    
input_dim = 1
hidden_dim = 50
layer_dim = 1
output_dim = 10
model_lstm = LSTMModel(input_dim, hidden_dim, layer_dim)

In [28]:
model_lstm = LSTMModel(input_dim=10, hidden_dim=100, layer_dim=1)
loss = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.001, momentum=0.01)

In [29]:
X_tr = X_tr.view(len(X_tr), 1, -1)
Y_tr = Y_tr.view(len(Y_tr), -1)

In [30]:
for epoch in range(num_epochs):
    train_iters, train_passed  = 0, 0
    train_loss, train_acc = 0., 0.
    start=time.time()
    model_lstm.train()

    optimizer.zero_grad()
    y_pred = model_lstm.forward(X_tr)
    y_pred = y_pred.view(len(Y_tr), -1)
    l = loss(y_pred, Y_tr)
    l.backward()
    optimizer.step()

    train_loss += l.item()
    train_acc += (y_pred.argmax(dim=0) == Y_tr).sum().item()

    train_iters += 1
    train_passed += len(X_tr)
    
    
    test_iters, test_passed  = 0, 0
    test_loss, test_acc = 0., 0.
    model_lstm.eval()
    with torch.no_grad():
        pred = model_lstm(X_ts)
        l = loss(pred, Y_ts)
        test_loss += l.item()
        test_acc += (y_pred.argmax(dim=0)==Y_ts).sum().item()
        test_iters += 1
        test_passed += len(X_ts)
        
    print("epoch: {}, taked: {:.3f}, train_loss: {}, train_acc: {}, test_loss: {}, test_acc: {}".format(
        epoch, time.time() - start, round(train_loss / train_iters, 3), round(train_acc / train_passed, 3),
        round(test_loss / test_iters, 3), round(test_acc / test_passed, 3))
    )

epoch: 0, taked: 0.035, train_loss: 103.703, train_acc: 0.253, test_loss: 103.825, test_acc: 0.48
epoch: 1, taked: 0.000, train_loss: 103.703, train_acc: 0.253, test_loss: 103.825, test_acc: 0.48
epoch: 2, taked: 0.016, train_loss: 103.703, train_acc: 0.253, test_loss: 103.825, test_acc: 0.48
epoch: 3, taked: 0.000, train_loss: 103.703, train_acc: 0.253, test_loss: 103.825, test_acc: 0.48
epoch: 4, taked: 0.000, train_loss: 103.703, train_acc: 0.253, test_loss: 103.825, test_acc: 0.48
epoch: 5, taked: 0.000, train_loss: 103.703, train_acc: 0.253, test_loss: 103.825, test_acc: 0.48
epoch: 6, taked: 0.000, train_loss: 103.703, train_acc: 0.253, test_loss: 103.825, test_acc: 0.48
epoch: 7, taked: 0.017, train_loss: 103.703, train_acc: 0.253, test_loss: 103.825, test_acc: 0.48
epoch: 8, taked: 0.002, train_loss: 103.703, train_acc: 0.253, test_loss: 103.825, test_acc: 0.48
epoch: 9, taked: 0.000, train_loss: 103.703, train_acc: 0.253, test_loss: 103.825, test_acc: 0.48
epoch: 10, taked: 0.